### SparkSession

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from utils.params import SCHEMA, URL_PARAMETROS, URL_PERIODOS, PRODUTOS_BKP
from utils.helper import *


spark = (
    SparkSession.builder
    .appName("default")
    .getOrCreate()
)

#https://www.bcb.gov.br/api/servico/sitebcb/historicotaxajurosdiario/TodosCampos?filtro=(codigoSegmento%20eq%20%272%27)%20and%20(codigoModalidade%20eq%20%27211101%27)%20and%20(InicioPeriodo%20eq%20%272025-03-24%27)

### Parâmetros

In [ ]:
PRODUTOS = obter_produtos_bacen(URL_PARAMETROS)
PRODUTOS = PRODUTOS if PRODUTOS else PRODUTOS_BKP

df_produtos = (
    spark.createDataFrame(PRODUTOS)
    .selectExpr(
        "codigoModalidade as CD_MODL",
        "Modalidade as DS_MODL",
        "codigoSegmento as CD_SEGM",
        "Segmento as NM_SEGM",
        "tipoModalidade as TP_MODL",
    )
    .withColumn("DS_TIPO_PERI_MODL",
                F.when(
                    F.col("TP_MODL") == "M", F.lit("Mensal")
                )
                .when(
                    F.col("TP_MODL") == "D", F.lit("Diária")
                )
                .when(
                    F.col("TP_MODL") == "A", F.lit("Anual")
                )
                .otherwise(
                    F.lit(None)
                )
    )
    .withColumn("dat_ref_carga", F.lit(date.today()))
    .withColumn("dh_exec", F.lit(datetime.now()))
)
df_produtos.show()

### Períodos

In [ ]:
PERIODOS, PERIODOS_INGEST = obter_periodos_bacen(URL_PERIODOS)

print(f"{PERIODOS['inicio']}")
print(f"{PERIODOS['final']}")

df_periodos = (
    spark.createDataFrame(PERIODOS_INGEST)
    .selectExpr(
        "Periodo as DS_PERI",
        "InicioPeriodo as DT_INIC_PERI",
        "FimPeriodo as DT_FINA_PERI",
        "tipoModalidade as TP_MODL",
    )
    .withColumn("dat_ref_carga", F.lit(date.today()))
    .withColumn("dh_exec", F.lit(datetime.now()))
)

df_periodos.show()

### Data única

In [ ]:

df_list = extrair_dados_bacen_data_unica(PRODUTOS, PERIODOS, data_alvo="2025-03-26")

df = spark.createDataFrame(df_list, schema=SCHEMA)

In [ ]:
df.show(truncate=False)

In [ ]:
df.select("QT_DIA_APROX").distinct().show(truncate=False)

In [ ]:
import pyspark.sql.functions as F

df.filter(F.col("NR_POSI") == 1).show(truncate=False)

### Mês específico

In [ ]:

df_list = extrair_dados_bacen_mes(PRODUTOS, PERIODOS, mes="2025-01")

df = spark.createDataFrame(df_list, schema=SCHEMA)

In [ ]:
df.show(truncate=False)

In [ ]:
df.count()

In [ ]:
df.select("dat_ref_carga").distinct().show(truncate=False)

### Ano específico

In [ ]:
df_list = extrair_dados_bacen_ano(PRODUTOS, PERIODOS, ano="2025")

df = spark.createDataFrame(df_list, schema=SCHEMA)

In [ ]:
df.show(truncate=False)

### Ano específico com range de meses

In [ ]:
df_list = extrair_dados_bacen_ano(PRODUTOS, ano="2024", mes_final = 3)

df = spark.createDataFrame(df_list, schema=SCHEMA)

In [ ]:
df.show(truncate=False)